In [ ]:
# -*- coding: utf-8 -*-
import os
os.environ["LC_ALL"] = "en_US.utf8"
os.environ["LANG"] = "en_US.utf8"

In [ ]:
# Install necessary packages
!pip install pandas
!pip install scikit-learn
!pip install numpy

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


1. Increase the Training Data: Adding more labeled sentences to your dataset can help the model learn more patterns and improve its performance. Consider expanding your dataset with additional labeled sentences to enhance the model's accuracy.


In [ ]:
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn import svm
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split

# Load the CSV file from a URL
url = "https://raw.githubusercontent.com/Cartesian1671/G4HSP-AI/main/corpus/Labeled_Documents/10xLabeled%20Dataset%20_without_apostrophes.csv"

data = pd.read_csv(url)

# The 'corpus' is the 'Sentence' column and the 'labels' is the 'Label' column
corpus = data['Sentence'].tolist()
labels = data['Label'].tolist()

# This vectorizer will convert our documents into a matrix of TF-IDF features
vectorizer = TfidfVectorizer()

# Vectorize our documents
tfidf_matrix = vectorizer.fit_transform(corpus)

# Convert these textual labels into numeric data
le = LabelEncoder()
numeric_labels = le.fit_transform(labels)

# Create an SVM model
model = svm.SVC()

# Split the data into training and test sets
X_train, X_test, y_train, y_test = train_test_split(tfidf_matrix, numeric_labels, test_size=0.2, random_state=42)

# Train the model on our data
model.fit(X_train, y_train)

# Make predictions on the test set and evaluate the model
y_pred = model.predict(X_test)

# Calculate the accuracy of the model
accuracy = (y_pred == y_test).mean()
print(f'Model accuracy: {accuracy}')


HTTPError: ignored

2. Fine-tune the Model: Experiment with different hyperparameters of the SVM model to find the optimal settings. You can try adjusting the kernel type, regularization parameter (C), or gamma value to see if they have any positive impact on the accuracy.

Explanation : * In this script, the CSV file is loaded from the provided URL, and the 'Sentence' and 'Label' columns are extracted into the 'corpus' and 'labels' variables, respectively. The labels are then converted into numeric form using LabelEncoder.

The data is split into training and test sets using train_test_split, with 80% for training and 20% for testing. The training data is vectorized using TfidfVectorizer, and the test data is vectorized using the fitted vectorizer.

A support vector machine (SVM) model is created using svm.SVC(). The hyperparameter grid is defined with different values for the 'kernel', 'C', and 'gamma' parameters. The grid search is performed using GridSearchCV, and the best hyperparameters and model are obtained.

Finally, the model is evaluated on the test set by making predictions and calculating the accuracy. The best model accuracy and hyperparameters are printed out.

 Run this script to fine-tune the SVM model with different hyperparameters and evaluate its performance on the test set.*

In [ ]:
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn import svm
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.model_selection import GridSearchCV

# Load the CSV file from a URL
url = "https://raw.githubusercontent.com/Cartesian1671/G4HSP-AI/main/corpus/Labeled_Documents/Labeled%20Dataset%20_without_apostrophes.csv"

data = pd.read_csv(url)

# The 'corpus' is the 'Sentence' column and the 'labels' is the 'Label' column
corpus = data['Sentence'].tolist()
labels = data['Label'].tolist()

# Convert labels into numeric data
le = LabelEncoder()
numeric_labels = le.fit_transform(labels)

# Split the data into training and test sets
X_train, X_test, y_train, y_test = train_test_split(corpus, numeric_labels, test_size=0.2, random_state=42)

# Create a TfidfVectorizer
vectorizer = TfidfVectorizer()

# Vectorize the training data
X_train = vectorizer.fit_transform(X_train)

# Vectorize the test data
X_test = vectorizer.transform(X_test)

# Create an SVM model
model = svm.SVC()

# Define the hyperparameter grid for grid search
param_grid = {
    'kernel': ['linear', 'poly', 'rbf', 'sigmoid'],
    'C': [0.1, 1, 10],
    'gamma': ['scale', 'auto']
}

# Perform grid search to find the optimal hyperparameters
grid_search = GridSearchCV(model, param_grid, cv=5)
grid_search.fit(X_train, y_train)

# Get the best hyperparameters and model
best_params = grid_search.best_params_
best_model = grid_search.best_estimator_

# Evaluate the model on the test set
y_pred = best_model.predict(X_test)
accuracy = accuracy_score(y_test, y_pred)
print(f"Best Model Accuracy: {accuracy}")
print(f"Best Hyperparameters: {best_params}")


/usr/local/lib/python3.10/dist-packages/sklearn/model_selection/_split.py:700: UserWarning: The least populated class in y has only 1 members, which is less than n_splits=5.
  warnings.warn(


Best Model Accuracy: 0.7741935483870968
Best Hyperparameters: {'C': 10, 'gamma': 'scale', 'kernel': 'linear'}


In [ ]:
data.head()

,Sentence,Label
0,A more direct evidence is that Hsp70 facilitat...,hsp-viral_interactions
1,"After ligand binding, TLRs dimerize and underg...",immunology
2,"After treated with Hsp90 inhibitors, HSV polym...",hsp-viral_interactions
3,Although Hsp70 inhibitors suppress RSV polymer...,hsp-viral_interactions
4,"Among these strategies, HBV infection increase...",immunology


In [ ]:
data.Label.unique()

array(['hsp-viral_interactions', 'immunology', 'viral_details',
       'virology', 'hsp-function', 'hsp-research_overview',
       'immune_system', 'antiviral_research', 'hsp-research',
       'domains containing JDP protein, p58IPK.'], dtype=object)

Save these parameters and load them into the model to use in the next stage of an attempt to improve the model further with Feature Engineering: 

In [ ]:
# Save the best hyperparameters to a file
import json

best_params_file = 'best_hyperparameters.json'
with open(best_params_file, 'w') as f:
    json.dump(best_params, f)


In [ ]:
# Load the best hyperparameters from the file
import json

best_params_file = 'best_hyperparameters.json'
with open(best_params_file, 'r') as f:
    best_params = json.load(f)

# Use the loaded hyperparameters in model training
model = svm.SVC(**best_params)


3. Feature Engineering: Explore different techniques to enhance the features used by the model. For instance, you can try n-grams (capturing word sequences) instead of single words, incorporate part-of-speech tags, or use other domain-specific features that may be relevant to your classification task.

In [ ]:
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn import svm
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.model_selection import GridSearchCV

# Load the CSV file from a URL
url = "https://raw.githubusercontent.com/Cartesian1671/G4HSP-AI/main/corpus/Labeled_Documents/Labeled%20Dataset%20_without_apostrophes.csv"

data = pd.read_csv(url)

# The 'corpus' is the 'Sentence' column and the 'labels' is the 'Label' column
corpus = data['Sentence'].tolist()
labels = data['Label'].tolist()

# Convert labels into numeric data
le = LabelEncoder()
numeric_labels = le.fit_transform(labels)

# Split the data into training and test sets
X_train, X_test, y_train, y_test = train_test_split(corpus, numeric_labels, test_size=0.2, random_state=42)

# Load the best hyperparameters from a file
import json

best_params_file = 'best_hyperparameters.json'
with open(best_params_file, 'r') as f:
    best_params = json.load(f)

# Create a TfidfVectorizer with n-grams
vectorizer = TfidfVectorizer(ngram_range=(1, 2))

# Vectorize the training data
X_train = vectorizer.fit_transform(X_train)

# Vectorize the test data
X_test = vectorizer.transform(X_test)

# Create an SVM model with the best hyperparameters
model = svm.SVC(**best_params)

# Train the model on the training data
model.fit(X_train, y_train)

# Make predictions on the test set
y_pred = model.predict(X_test)

# Evaluate the model on the test set
accuracy = accuracy_score(y_test, y_pred)
print(f"Model Accuracy: {accuracy}")


Model Accuracy: 0.7741935483870968


Explanation: * the TfidfVectorizer already incorporates some feature engineering techniques by considering the TF-IDF values of individual words. However, thee accuracy of the model did not improve.  For this reason we need to enhance the features by incorporating n-grams, part-of-speech tags, or other domain-specific features.

To incorporate n-grams in the TfidfVectorizer, you can set the ngram_range parameter to capture word sequences of different lengths. For example:
we used first
vectorizer = TfidfVectorizer(ngram_range=(1, 2))
and then chnged it to (1, 10)

This will consider both single words and word pairs (bigrams) as features.

For incorporating part-of-speech tags and  domain-specific features for G-quadruplex and stress proteins, we  add an additonal preprocessing of the corpus text and extract those features separately. Then, we  combine those features with the TF-IDF features before training the model.*

In [ ]:
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn import svm
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.model_selection import GridSearchCV

# Load the CSV file from a URL
url = "https://raw.githubusercontent.com/Cartesian1671/G4HSP-AI/main/corpus/Labeled_Documents/Labeled%20Dataset%20_without_apostrophes.csv"

data = pd.read_csv(url)

# The 'corpus' is the 'Sentence' column and the 'labels' is the 'Label' column
corpus = data['Sentence'].tolist()
labels = data['Label'].tolist()

# Convert labels into numeric data
le = LabelEncoder()
numeric_labels = le.fit_transform(labels)

# Split the data into training and test sets
X_train, X_test, y_train, y_test = train_test_split(corpus, numeric_labels, test_size=0.2, random_state=42)

# Load the best hyperparameters from a file
import json

best_params_file = 'best_hyperparameters.json'
with open(best_params_file, 'r') as f:
    best_params = json.load(f)

# Create a TfidfVectorizer with n-grams
vectorizer = TfidfVectorizer(ngram_range=(1,2))
# Vectorize the training data
X_train = vectorizer.fit_transform(X_train)

# Vectorize the test data
X_test = vectorizer.transform(X_test)

# Create an SVM model with the best hyperparameters
model = svm.SVC(**best_params)

# Train the model on the training data
model.fit(X_train, y_train)

# Make predictions on the test set
y_pred = model.predict(X_test)

# Evaluate the model on the test set
accuracy = accuracy_score(y_test, y_pred)
print(f"Model Accuracy: {accuracy}")


Model Accuracy: 0.7741935483870968


Text preprocessing:
To preprocess the text and extract additional features like part-of-speech tags or other domain-specific features, you can use libraries such as NLTK. Here's an example of how you can use NLTK for part-of-speech tagging:

Install NLTK library (if not already installed) by running pip install nltk in your terminal.

Import the necessary NLTK modules in your Python script:

In [ ]:
import nltk
from nltk import pos_tag
from nltk.tokenize import word_tokenize


Download the required NLTK data by running the following code once:


In [ ]:
nltk.download('punkt')
nltk.download('averaged_perceptron_tagger')


[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Unzipping taggers/averaged_perceptron_tagger.zip.


True

Define a function to preprocess your text and extract the additional features. In this case, we'll focus on part-of-speech tagging:

In [ ]:
def preprocess(text):
    # Tokenize the text into individual words
    tokens = word_tokenize(text)
    
    # Perform part-of-speech tagging
    pos_tags = pos_tag(tokens)
    
    # Extract only the part-of-speech tags
    tags = [tag for _, tag in pos_tags]
    
    return tags


Apply the preprocess function to your text data to extract the part-of-speech tags:  The pos_tags variable will contain a list of lists, where each inner list represents the part-of-speech tags for each sentence in the corpus.


In [ ]:
corpus = [
    'G-quadruplexes are non-canonical nucleic acids secondary structures.',
    'They are found in regulatory genomic regions.',
    # Add more sentences as needed
]

# Apply preprocessing to extract part-of-speech tags
pos_tags = [preprocess(text) for text in corpus]


Run the newly optimized SVM again but now with the pos_tags included 

Now we can run our model and check the accuracy

In [ ]:
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn import svm
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.model_selection import GridSearchCV

# Load the CSV file from a URL
url = "https://raw.githubusercontent.com/Cartesian1671/G4HSP-AI/main/corpus/Labeled_Documents/Labeled%20Dataset%20_without_apostrophes.csv"

data = pd.read_csv(url)

# The 'corpus' is the 'Sentence' column and the 'labels' is the 'Label' column
corpus = data['Sentence'].tolist()
labels = data['Label'].tolist()

# Convert labels into numeric data
le = LabelEncoder()
numeric_labels = le.fit_transform(labels)

# Split the data into training and test sets
X_train, X_test, y_train, y_test = train_test_split(corpus, numeric_labels, test_size=0.2, random_state=42)

# Apply part-of-speech tagging and extract pos_tags
pos_tags_train = [preprocess(text) for text in X_train]
pos_tags_test = [preprocess(text) for text in X_test]

# Combine TF-IDF features with part-of-speech tags
X_train_combined = []
for i in range(len(X_train)):
    features = X_train[i] + pos_tags_train[i]
    X_train_combined.append(' '.join(features))

X_test_combined = []
for i in range(len(X_test)):
    features = X_test[i] + pos_tags_test[i]
    X_test_combined.append(' '.join(features))

# Create a TfidfVectorizer with n-grams
vectorizer = TfidfVectorizer(ngram_range=(5, 9))
# Vectorize the training data
X_train_vectorized = vectorizer.fit_transform(X_train_combined)

# Vectorize the test data
X_test_vectorized = vectorizer.transform(X_test_combined)

# Load the best hyperparameters from a file
import json

best_params_file = 'best_hyperparameters.json'
with open(best_params_file, 'r') as f:
    best_params = json.load(f)

# Create an SVM model with the best hyperparameters
model = svm.SVC(**best_params)

# Train the model on the training data
model.fit(X_train_vectorized, y_train)

# Make predictions on the test set
y_pred = model.predict(X_test_vectorized)

# Evaluate the model on the test set
accuracy = accuracy_score(y_test, y_pred)
print(f"Model Accuracy: {accuracy}")


TypeError: ignored

4. Handle Class Imbalance: If your dataset has imbalanced classes, where some labels have significantly fewer examples than others, it may lead to biased predictions. Techniques such as oversampling the minority class or undersampling the majority class can help balance the data and potentially improve accuracy.

There are two ways of doing this:   oversampling or undersampling techniques. 

Oversampling:
 Oversampling involves increasing the number of instances in the minority class to balance it with the majority class.  Method: by replicating existing instances from the minority class or by generating synthetic samples using techniques like SMOTE (Synthetic Minority Over-sampling Technique). Result: Oversampling provide more training data for the minority class and can prevent the model from being biased towards the majority class.

Undersampling: 
Undersampling involves reducing the number of instances in the majority class to balance it with the minority class. Method:  by randomly removing instances from the majority class until the desired balance is achieved. Result: Undersampling helps reduce the dominance of the majority class and allows the model to pay more attention to the minority class.

This is the script:


In [ ]:
import numpy as np
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn import svm
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from imblearn.over_sampling import RandomOverSampler
from imblearn.under_sampling import RandomUnderSampler
from sklearn.metrics import accuracy_score

# Load the CSV file from a URL
url = "https://raw.githubusercontent.com/Cartesian1671/G4HSP-AI/main/corpus/Labeled_Documents/Labeled%20Dataset%20_without_apostrophes.csv"
data = pd.read_csv(url)

# The 'corpus' is the 'Sentence' column and the 'labels' is the 'Label' column
corpus = data['Sentence'].tolist()
labels = data['Label'].tolist()

# Convert labels into numeric data
le = LabelEncoder()
numeric_labels = le.fit_transform(labels)

# Split the data into training and test sets
X_train, X_test, y_train, y_test = train_test_split(corpus, numeric_labels, test_size=0.2, random_state=42)

# Convert X_train to a 2D array
X_train = np.array(X_train).reshape(-1, 1)

# Apply oversampling with RandomOverSampler
oversampler = RandomOverSampler()
X_train_oversampled, y_train_oversampled = oversampler.fit_resample(X_train, y_train)

# Apply undersampling with RandomUnderSampler
undersampler = RandomUnderSampler()
X_train_undersampled, y_train_undersampled = undersampler.fit_resample(X_train, y_train)

# Create a TfidfVectorizer
vectorizer = TfidfVectorizer()

# Vectorize the training data
X_train_tfidf = vectorizer.fit_transform(X_train_oversampled[:, 0])

# Vectorize the test data
X_test_tfidf = vectorizer.transform(X_test)

# Create an SVM model
model = svm.SVC()

# Train the model on the oversampled training data
model.fit(X_train_tfidf, y_train_oversampled)

# Make predictions on the test set
y_pred = model.predict(X_test_tfidf)

# Evaluate the model on the test set
accuracy = accuracy_score(y_test, y_pred)
print(f"Model Accuracy: {accuracy}")


Model Accuracy: 0.7419354838709677



5. Cross-Validation: Perform cross-validation to get a more reliable estimate of your model's performance. Split your data into multiple training and validation sets and evaluate the model on each fold to obtain a more robust accuracy estimation.


Explanation: To perform cross-validation and obtain a more robust accuracy estimation, you can use the cross_val_score function from scikit-learn.  we use the cross_val_score function to perform 5-fold cross-validation. The cv parameter specifies the number of folds (in this case, 5). The cross-validation scores are then averaged to obtain an average accuracy. By using cross-validation, you get a more reliable estimate of your model's performance on unseen data.

In [ ]:
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn import svm
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from imblearn.over_sampling import RandomOverSampler
from imblearn.under_sampling import RandomUnderSampler
from sklearn.model_selection import cross_val_score

# Load the CSV file from a URL
url = "https://raw.githubusercontent.com/Cartesian1671/G4HSP-AI/main/corpus/Labeled_Documents/Labeled%20Dataset%20_without_apostrophes.csv"
data = pd.read_csv(url)

# The 'corpus' is the 'Sentence' column and the 'labels' is the 'Label' column
corpus = data['Sentence'].tolist()
labels = data['Label'].tolist()

# Convert labels into numeric data
le = LabelEncoder()
numeric_labels = le.fit_transform(labels)

# Create a TfidfVectorizer
vectorizer = TfidfVectorizer()

# Vectorize the data
X = vectorizer.fit_transform(corpus)

# Apply oversampling with RandomOverSampler
oversampler = RandomOverSampler()
X_oversampled, y_oversampled = oversampler.fit_resample(X, numeric_labels)

# Apply undersampling with RandomUnderSampler
undersampler = RandomUnderSampler()
X_resampled, y_resampled = undersampler.fit_resample(X_oversampled, y_oversampled)

# Create an SVM model
model = svm.SVC()

# Perform cross-validation
cv_scores = cross_val_score(model, X_resampled, y_resampled, cv=5)

# Calculate the average accuracy from cross-validation scores
avg_accuracy = cv_scores.mean()

print(f"Average Accuracy: {avg_accuracy}")


Average Accuracy: 1.0


Result says Average Accuracy: 1.0.  This is possibly an indication of an issue in the dataset which we will investigate but first chack other performance metrics: 

In [ ]:
import numpy as np
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn import svm
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from imblearn.over_sampling import RandomOverSampler
from imblearn.under_sampling import RandomUnderSampler
from sklearn.metrics import accuracy_score

# Load the CSV file from a URL
url = "https://raw.githubusercontent.com/Cartesian1671/G4HSP-AI/main/corpus/Labeled_Documents/Labeled%20Dataset%20_without_apostrophes.csv"
data = pd.read_csv(url)

# The 'corpus' is the 'Sentence' column and the 'labels' is the 'Label' column
corpus = data['Sentence'].tolist()
labels = data['Label'].tolist()

# Convert labels into numeric data
le = LabelEncoder()
numeric_labels = le.fit_transform(labels)

# Split the data into training and test sets
X_train, X_test, y_train, y_test = train_test_split(corpus, numeric_labels, test_size=0.2, random_state=42)

# Convert X_train to a 2D array
X_train = np.array(X_train).reshape(-1, 1)

# Apply oversampling with RandomOverSampler
oversampler = RandomOverSampler()
X_train_oversampled, y_train_oversampled = oversampler.fit_resample(X_train, y_train)

# Apply undersampling with RandomUnderSampler
undersampler = RandomUnderSampler()
X_train_undersampled, y_train_undersampled = undersampler.fit_resample(X_train, y_train)

# Create a TfidfVectorizer
vectorizer = TfidfVectorizer()

# Vectorize the training data
X_train_tfidf = vectorizer.fit_transform(X_train_oversampled[:, 0])

# Vectorize the test data
X_test_tfidf = vectorizer.transform(X_test)

# Create an SVM model
model = svm.SVC()

# Train the model on the oversampled training data
model.fit(X_train_tfidf, y_train_oversampled)

# Make predictions on the test set
y_pred = model.predict(X_test_tfidf)

# Evaluate the model on the test set
accuracy = accuracy_score(y_test, y_pred)
print(f"Model Accuracy: {accuracy}")


Model Accuracy: 0.7419354838709677


In [ ]:
import numpy as np
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn import svm
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
from imblearn.over_sampling import RandomOverSampler
from imblearn.under_sampling import RandomUnderSampler

# Load the CSV file from a URL
url = "https://raw.githubusercontent.com/Cartesian1671/G4HSP-AI/main/corpus/Labeled_Documents/Labeled%20Dataset%20_without_apostrophes.csv"
data = pd.read_csv(url)

# The 'corpus' is the 'Sentence' column and the 'labels' is the 'Label' column
corpus = data['Sentence'].tolist()
labels = data['Label'].tolist()

# Convert labels into numeric data
le = LabelEncoder()
numeric_labels = le.fit_transform(labels)

# Split the data into training and test sets
X_train, X_test, y_train, y_test = train_test_split(corpus, numeric_labels, test_size=0.2, random_state=42)

# Convert X_train to a 2D array
X_train = np.array(X_train).reshape(-1, 1)

# Apply oversampling with RandomOverSampler
oversampler = RandomOverSampler()
X_train_oversampled, y_train_oversampled = oversampler.fit_resample(X_train, y_train)

# Apply undersampling with RandomUnderSampler
undersampler = RandomUnderSampler()
X_train_undersampled, y_train_undersampled = undersampler.fit_resample(X_train, y_train)

# Create a TfidfVectorizer
vectorizer = TfidfVectorizer()

# Vectorize the training data
X_train_tfidf = vectorizer.fit_transform(X_train_oversampled[:, 0])

# Vectorize the test data
X_test_tfidf = vectorizer.transform(X_test)

# Create an SVM model
model = svm.SVC()

# Train the model on the oversampled training data
model.fit(X_train_tfidf, y_train_oversampled)

# Make predictions on the test set
y_pred = model.predict(X_test_tfidf)

# Calculate precision, recall, and F1-score
report = classification_report(y_test, y_pred, labels=np.unique(y_test), target_names=le.classes_)

print("Classification Report:")
print(report)


/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:2326: UserWarning: labels size, 7, does not match size of target_names, 10
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


Classification Report:
                                         precision    recall  f1-score   support

                     antiviral_research       0.00      0.00      0.00         1
domains containing JDP protein, p58IPK.       0.00      0.00      0.00         1
                           hsp-function       0.00      0.00      0.00         1
                           hsp-research       0.74      1.00      0.85        23
                  hsp-research_overview       0.00      0.00      0.00         1
                 hsp-viral_interactions       0.00      0.00      0.00         1
                          immune_system       0.00      0.00      0.00         3

                               accuracy                           0.74        31
                              macro avg       0.11      0.14      0.12        31
                           weighted avg       0.55      0.74      0.63        31



/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


6. Model Selection: Consider exploring other machine learning algorithms apart from SVM, such as Random Forest, Gradient Boosting, or Neural Networks. Different models may perform better depending on the specific characteristics of your dataset.